In [4]:
# brew install graphviz
%pip install graphviz
from graphviz import Digraph

Note: you may need to restart the kernel to use updated packages.


In [5]:
from micrograd.engine import Value

In [6]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [7]:
# a very simple example
x = Value(1.0)
y = (x * 2 + 1).relu()
y.backward()
#draw_dot(y)

In [8]:
# a simple 2D neuron
import random
from micrograd import nn

random.seed(1337)
n = nn.Neuron(2)
x = [Value(1.0), Value(-2.0)]
y = n(x)
y.backward()

#dot = draw_dot(y)
#dot

In [9]:
from micrograd.nn import MLP
n = MLP(3,[4,4,1])

In [10]:
xs = [
    [2.0, 3.0,-1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0,1.0],
    [1.0, 1.0,-1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [n(x) for x in xs]
ypred

[Value(data=0.17491833012255345, grad=0),
 Value(data=-1.9336279665121072, grad=0),
 Value(data=-0.42706587737738166, grad=0),
 Value(data=0.15975564404725467, grad=0)]

In [11]:
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=2.5866850283970813, grad=0)

In [12]:
loss.backward()

In [13]:
#draw_dot(loss)

In [14]:
#dot.render('gout')

Training av very simple model: y = wx + b
Using micrograd MLP and a dataset of 2 samples

In [15]:
# Instantiating a 1-layer MLP and setting its parameters
n = MLP(1,[1])   # 1 input, 1 output, y = wx + b
params = n.parameters()
params[0].data = 1.9
params[1].data = 2.1
for p in params:
    print(p.data)
print(n)

# Loading the dataset consisting 2 samples
xs = [[2.0],
      [3.0]]
ys = [6.0,8.0]

1.9
2.1
MLP of [Layer of [LinearNeuron(1)]]


In [16]:
# Generating predictions
ypred = [n(x) for x in xs]
print([yp.data for yp in ypred])

[5.9, 7.799999999999999]


In [17]:
# Calculating the loss
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=0.05000000000000035, grad=0)

In [18]:
n.zero_grad()
loss.backward()
learning_rate = 0.01
for p in n.parameters():
    p.data -= p.grad * learning_rate
    print(p.data)

1.916
2.1060000000000003


In [19]:
#draw_dot(loss)

In [20]:
for i in range(10000):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    n.zero_grad()
    loss.backward()
    for p in n.parameters():
        p.data -= p.grad * learning_rate
    if i % 1000 == 0:
        print(loss, params[0].data, params[1].data)

Value(data=0.025160000000000158, grad=1) 1.9272399999999998 2.1101600000000005
Value(data=7.656090196463862e-05, grad=1) 1.9878236470835418 2.0314954381628723
Value(data=5.247050521673717e-06, grad=1) 1.9968123448689865 2.008245210676154
Value(data=3.5960311948400426e-07, grad=1) 1.9991655017471992 2.0021585189176463
Value(data=2.46451607446296e-08, grad=1) 1.999781536174616 2.0005650800326187
Value(data=1.6890397086579587e-09, grad=1) 1.9999428082169834 2.0001479326591287
Value(data=1.1575721361195228e-10, grad=1) 1.9999850277269529 2.0000387273843936
Value(data=7.933343684492234e-12, grad=1) 1.9999960803991699 2.0000101384664535
Value(data=5.437064364293277e-13, grad=1) 1.9999989738852193 2.000002654155546
Value(data=3.726255911042191e-14, grad=1) 1.9999997313727624 2.00000069483306


Training av very simple model: y = wx + b
Using micrograd and a dataset of 2 samples

In [21]:
# Dataset
x = [[Value(2.0)],
     [Value(3.0)]]
y = [Value(6.0),
     Value(8.0)]

# Weights initialization
w = [Value(1.9)]
b = Value(2.1)

In [22]:

ypred = [sum((wi*xi for wi,xi in zip(w, xt)), b) for xt in x]
ypred

[Value(data=5.9, grad=0), Value(data=7.799999999999999, grad=0)]

In [23]:
# Calculating the loss
loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))
print(f'loss: {loss.data} | grad: {loss.grad}')

loss: 0.05000000000000035 | grad: 0


In [24]:
# Reset gradients
for wi in w:
    wi.grad = 0
b.grad = 0

# Backpropagation - calculating gradients
loss.backward()

# Updating weights
learning_rate = 0.01
for wi in w:
    wi.data -= wi.grad * learning_rate
b.data -= b.grad * learning_rate
print(f'w:{[wi.data for wi in w]}, b:{b.data}')

w:[1.916], b:2.1060000000000003


In [136]:
for i in range(1000):
    ypred = [sum((wi*xi for wi,xi in zip(w, xt)), b) for xt in x]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))
    for wi in w:
        wi.grad = 0
    b.grad = 0
    loss.backward()
    for wi in w:
        wi.data -= wi.grad * learning_rate
    b.data -= b.grad * learning_rate
    if i % 100 == 0:
        print(f'loss: {loss.data} w:{[wi.data for wi in w]}, b:{b.data}')

loss: 2.506518044405186e-08 w:[1.9997796824343979], b:2.0005698749298104
loss: 1.917174553094562e-08 w:[1.99980731649646], b:2.000498396474904
loss: 1.4664000824692564e-08 w:[1.9998314844645497], b:2.0004358834424942
loss: 1.1216136779807323e-08 w:[1.999852621085011], b:2.0003812113147013
loss: 8.578949617396645e-09 w:[1.9998711065746826], b:2.000333396620033
loss: 6.5618294412276236e-09 w:[1.9998872734604456], b:2.0002915792421763
loss: 5.0189833877748825e-09 w:[1.999901412560892], b:2.0002550069477585
loss: 3.838898050240777e-09 w:[1.999913778217727], b:2.00022302185478
loss: 2.9362795412408877e-09 w:[1.9999245928710025], b:2.0001950485982682
loss: 2.2458886461229267e-09 w:[1.9999340510604886], b:2.0001705839803217
